In [88]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse
from datetime import datetime

In [147]:
import re 

def has_cyrillic(text):
    return bool(re.search('[а-яА-Я]', text))

def fix_encoding(page):
    for encoding in ['ISO-8859-1', 'utf-8', 'windows-1251']:
        page.encoding = encoding
        soup = BeautifulSoup(page.text, 'html.parser')
        title = soup.find_all('h2')[0].text
        if has_cyrillic(title):
            return encoding
    return None

In [ ]:
def get_event_sportsmens(title, date, place, online_table_link):
    if online_table_link is None:
        return None
    df = pd.DataFrame(columns=['title','date','place','online_link','category', 'segment', 'position', 'name'])
    page = requests.get(online_table_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    all_off = soup.find_all('a', {'href': re.compile(r'SEG[0-9]{3}OF')})
    qty = 0
    for link in all_off:
        off_link = urllib.parse.urljoin(online_table_link, link.get('href'))
        page = requests.get(off_link)
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'html.parser')
            if len(soup.find_all('h2')) < 2:
                continue
            category, segment = [x.strip() for x in soup.find_all('h2')[1].text.split('-')[:2]]
            lines = soup.find_all('tr')
            for line in lines[1:]:
                cells = line.find_all('td')
                if cells[0].text != '\xa0':
                    df.loc[len(df)] = [title, date, place, off_link, category, segment, cells[0].text, cells[1].text]
    return df

In [161]:
def get_event_judges(title, date, place, online_table_link):
    if online_table_link is None:
        return None
    df = pd.DataFrame(columns=['title','date','place','online_link','category', 'segment', 'position', 'name'])
    page = requests.get(online_table_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    all_off = soup.find_all('a', {'href': re.compile(r'SEG[0-9]{3}OF')})
    qty = 0
    for link in all_off:
        off_link = urllib.parse.urljoin(online_table_link, link.get('href'))
        page = requests.get(off_link)
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'html.parser')
            if len(soup.find_all('h2')) < 2:
                continue
            category, segment = [x.strip() for x in soup.find_all('h2')[1].text.split('-')[:2]]
            lines = soup.find_all('tr')
            for line in lines[1:]:
                cells = line.find_all('td')
                if cells[0].text != '\xa0':
                    df.loc[len(df)] = [title, date, place, off_link, category, segment, cells[0].text, cells[1].text]
    return df

In [186]:
def parse_event_info(page, encoding):
    page = requests.get(online_link)
    page.encoding = encoding
    soup = BeautifulSoup(page.text, 'html.parser')
    h2s = soup.find_all('h2')
    title = h2s[0].text
    h3s = soup.find_all('h3')
    place = ''
    date = h3s[1].text.split('-')[0].strip()
    all_res = soup.find_all('a', {'href': re.compile(r'CAT[0-9]{3}RS')})
    all_off = soup.find_all('a', {'href': re.compile(r'SEG[0-9]{3}OF')})
    event_id = len(df_event)
    df_event.loc[event_id] = [title, date, place, online_link]
    return event_id, all_res, all_off

In [185]:
def parse_category_info(df_cat, page, encoding, online_link):
    

In [180]:
def get_event_results(df_event, df_cat, df_cat_res, df_seg, df_off, online_table_link):

    online_link = online_table_link.rstrip()
    online_link = online_link if online_link[-1] == '/' else online_link + '/'
    page = requests.get(online_link)
    if page.status_code != 200:
        return None
    encoding = fix_encoding(page)
    
    event_id, all_res, all_off = parse_event_info(df_event, online_link, encoding)
    
    for link in all_res:
        res_link = urllib.parse.urljoin(online_link, link.get('href'))
        page = requests.get(res_link)
        if(page.encoding == 'ISO-8859-1'):
            page.encoding = 'utf-8'
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'html.parser')
            if len(soup.find_all('h2')) < 2:
                continue
            category = soup.find_all('h2')[1].text
            lines = soup.find_all('tr')
            for line in lines:
                cells = line.find_all('td')
                if len(cells) >= 4:
                    rank = cells[0].text
                    name = cells[1].a.text
                    club = cells[1].text[len(name):]
                    name = ' '.join([x.strip() for x in name.split()])
                    name = name.replace("ё", "е")
                    name = name.replace("Ё", "Е")
                    if len(cells) == 6:
                        points = cells[-3].text
                    else:
                        points = cells[-2].text
                    df.loc[len(df)] = [title, date, place, res_link, category, rank, name, club, points]
                    
    return df

In [181]:
df_event = pd.DataFrame(columns=['title','date','place','online_link'])
df_cat = pd.DataFrame(columns=['event_id', 'category', 'online_link'])
df_cat_res = pd.DataFrame(columns=['cat_id', 'rank', 'name', 'club', 'points'])
df_seg = pd.DataFrame(columns=['cat_id', 'segment'])
df_off = pd.DataFrame(columns=['seg_id', 'position', 'name'])

with open('event_list.txt', 'r') as file:
    for online_link in file:
        if online_table_link is not None:
            get_event_results(df_event, df_cat, df_cat_res, df_seg, df_off, online_link)

KeyboardInterrupt: 

In [183]:
df_event.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        27 non-null     object
 1   date         27 non-null     object
 2   place        27 non-null     object
 3   online_link  27 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB


In [184]:
df_event.head()

,title,date,place,online_link
0,IISF Cup 2021,25.06.2021,,https://skating-vision.online/events/2021/IISF...
1,Открытое первенство «Невский конёк»,14.03.2021,,https://skating-vision.online/events/2021/alek...
2,Соревнования «Первый снег»,18.11.2020,,https://skating-vision.online/events/2021/aren...
3,ПЕРВЕНСТВО ПРИМОРСКОГО КРАЯ,23.01.2021,,https://skating-vision.online/events/2021/artem/
4,ПЕРВЕНСТВО ПРИМОРСКОГО КРАЯ,23.01.2021,,https://skating-vision.online/events/2021/arte...


In [5]:
df.to_csv("ffkkmo_dump.csv")

In [45]:
df_judges.to_csv("df_judges_dump.csv")